In [6]:
import matplotlib
matplotlib.use("svg")
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use("ggplot")
from mpl_toolkits.axes_grid1 import ImageGrid
from scipy import spatial
from pandas import DataFrame as df
from datetime import datetime
import pandas as pd
import numpy as np
import collaborative_filtering as cf
import linear_regression as lr
import networkx as nx
import seaborn as sns
import movie_parsing
import process_data

links = pd.read_csv('./data/ml-latest-small/links.csv')
movies = pd.read_csv("./data/ml-latest-small/movies.csv")
ratings = pd.read_csv("./data/ml-latest-small/ratings.csv")
df_movies = pd.read_csv('./data/movie_objects.csv', encoding = 'latin1')

df_user = (pd.read_csv("./data/u.user",sep='|',names=['userId', 'age', 'gender', 'occupation', 'zip_code']))[:671]

In [7]:
df_movies

,cast,casting_director,cinematographer,countries,director,distributors,genres,imdbId,languages,plot,producer,production_company,rating,runtimes,stunt_performer,title,visual_effects,votes,writer,year
0,"[(0000158, Hanks, Tom), (0000741, Allen, Tim),...",NaN,NaN,[USA],"[(0005124, Lasseter, John)]","[(0044279, Buena Vista Pictures), (0302554, Bu...","[Animation, Adventure, Comedy, Family, Fantasy]",114709,[English],[A little boy named Andy loves to be in his ro...,"[(0036366, Arnold, Bonnie), (0146216, Catmull,...","[(0017902, Pixar Animation Studios), (0008970,...",8.3,NaN,NaN,Toy Story,"[(0011185, Adams, Mark), (0017492, Alden, Naro...",639800,"[(0005124, Lasseter, John), (0230032, Docter, ...",1995
1,"[(0000245, Williams, Robin), (0404993, Hyde, J...","[(0289408, Foy, Nancy)]","[(0005630, Ackerman, Thomas E.)]",[USA],"[(0002653, Johnston, Joe)]","[(0060005, Columbia TriStar Film), (0077851, C...","[Action, Adventure, Family, Fantasy, Thriller]",113497,"[English, French]",[After being trapped in a jungle board game fo...,"[(0181202, Cort, Robert W.), (0276059, Field, ...","[(0005883, TriStar Pictures), (0031772, Inters...",6.9,[104],"[(0027392, Anderson, Roy T.), (0028136, Andre,...",Jumanji,"[(0019016, Alexy, Philip Edward), (0028626, An...",212636,"[(0378144, Hensleigh, Jonathan), (0852430, Tay...",1995
2,"[(0000527, Matthau, Walter), (0000493, Lemmon,...","[(0276042, Howard-Field, Sharon)]","[(0005714, Fujimoto, Tak)]",[USA],"[(0222043, Deutch, Howard)]","[(0026840, Warner Bros.), (0072315, National B...","[Comedy, Romance]",113228,[English],[Things don't seem to change much in Wabasha C...,"[(0075828, Berman, Richard C.), (0204862, Davi...","[(0016761, Lancaster Gate), (0026840, Warner B...",6.6,[101],"[(0007200, Brewer, Charlie), (0132587, Campbel...",Grumpier Old Men,"[(0378333, Heo, Jongwoo), (0421117, Jennings, ...",19585,"[(0425756, Johnson, Mark Steven), (0425756, Jo...",1995
3,"[(0001365, Houston, Whitney), (0000291, Basset...","[(0113764, Brown, Jaki)]","[(0475746, Kurita, Toyomichi)]",[USA],"[(0001845, Whitaker, Forest)]","[(0000756, Twentieth Century Fox Film Corporat...","[Comedy, Drama, Romance]",114885,[English],[This story based on the best selling novel by...,"[(0060103, Bass, Ronald), (0433845, K, Caron),...","[(0000756, Twentieth Century Fox Film Corporat...",5.7,[124],"[(0404393, Hutchinson, Rawn), (0781009, Sebek,...",Waiting to Exhale,"[(0215199, deJong, Karen), (0504931, Leung, Da...",7561,"[(0573334, McMillan, Terry), (0573334, McMilla...",1995
4,"[(0000188, Martin, Steve), (0000473, Keaton, D...","[(0338528, Greenberg, Jeff), (0349304, Guthrie...","[(0204567, Davis, Elliot), (0005710, Fraker, W...",[USA],"[(0796124, Shyer, Charles)]","[(0044279, Buena Vista Pictures), (0037052, Am...","[Comedy, Family, Romance]",113041,[English],"[In this sequel to ""Father of the Bride"", Geor...","[(0062071, Baum, Carol), (0088692, Block, Bruc...","[(0005929, Sandollar Productions), (0019819, T...",5.9,[106],"[(0102334, Boyle, Shelley S.), (0123594, Burto...",Father of the Bride Part II,"[(0421117, Jennings, Brian), (0550408, Marscha...",27136,"[(0352443, Hackett, Albert), (0329304, Goodric...",1995
5,"[(0000199, Pacino, Al), (0000134, De Niro, Rob...","[(0863659, Timmermann, Bonnie)]","[(0005883, Spinotti, Dante)]",[USA],"[(0000520, Mann, Michael)]","[(0026840, Warner Bros.), (0021869, ELKE), (00...","[Action, Crime, Drama, Thriller]",113277,"[English, Spanish]",[Hunters and their prey--Neil and his professi...,"[(0115764, Brugge, Pieter Jan), (0149052, Cesa...","[(0026840, Warner Bros.), (0021592, Regency En...",8.2,[170],"[(0115397, Brubaker, Tony), (0171018, Coleman,...",Heat,"[(0008671, Abers, Mimi), (0307213, Gareri, Joe...",439967,"[(0000520, Mann, Michael)]",1995
6,"[(0000148, Ford, Harrison), (0000566, Ormond, ...","[(0748039, Rubin, David)]","[(0005850, Rotunno, Giuseppe)]","[Germany, USA]","[(0001628, Pollack, Sydney)]","[(0023400, Paramount Pictures), (0070627, Colu...","[Comedy, Drama]",114319,"[English

## Movie Recommendation -  Report

The goal of this project is to build a movie recommender using more than collaborative filtering. Movie attributes from thousands of movies will be collected and analyzed to make educated recommendations. For now the project can be broken down into three parts:
* **Dataset compilation** 
* **Visualization** 
* **Recommendation model**

### Datasets Compilation

##### MovieLens 
This project will make use of the typical recommendation systems dataset known as the MovieLens dataset as compiled by Grouplens. Due to computational limitations, the smallest dataset (last updated 10/2016) will be explored rather than the complete 20M ratings one. 

The smallest dataset contains 100,000 ratings and 1,300 tag applications applied to 9,000 movies by 700 users. The dataset consists of four .csv files: links.csv, linking movies to IMDb and TMDb; movies.csv, containing movie title and genres; tags.csv, containing a tag or keyword for the movie; and ratings.csv, which has the movie ratings by the users. In addition, the users data included in the complete dataset will be used, taking only the users relevant to the small dataset.

http://grouplens.org/datasets/movielens/

##### IMDb

The scope of the MovieLens dataset is for collaborative filtering but it sets the ground for expansion with the links.csv. This project will make use of the IMDb IDs that are linked to each rated movie to collect movie information from IMDb itself. For the meantime, the fields that will be collected from IMDb are the following: cast, casting_director, cinematographer, countries, director, distributors, genres, id, languages, plot, producer, production company, rating, runtimes, stunt performer, title, visual effects, votes, writer, and year.

Unfortunately IMDb does not offer an official API to extract its data. The data, which is publicly available, could have been scraped for this project but instead an unofficial API called IMDbPY will be used. Their website is http://imdbpy.sourceforge.net/ and the library can be installed through pip with "pip install IMDbPY"

IMDbPY can be used to create a "movie object" by specifying the IMDbID of the movie and then the desired fields can be retrieved from this object. For example, the code below initializes the API object and will create a movie object for the movie with the IMDbID 114709. 

In [ ]:
import imdb
imdbID = "114709"
imdb_api = imdb.IMDb()
movie_object = imdb_api.get_movie(imdbID)

In [ ]:
print movie_object['title']
print movie_object['genres']
print movie_object['plot'][0]

The algorithm to create the database then goes this way:

* Read all IMDbIDs from the links.csv file. 
* While iterating through every IMDbID, create a IMDbPY movie object and extract and store only desired fields.
* Dump results into .csv file for later reference.

Below is a screenshot of how the .csv looks like. Fields that may contain more than one element are stored in lists. Also, for fields that involve entities (actors, directors, etc...), the values were stored as a list of tuples with their names and respective IMDbIDs. The IMDbIDs were stored just in case for future reference. 

<img src="./fig/output.jpeg">

###  Visualization

We began with getting an overview of the data. 

In [ ]:
movieRating=ratings.merge(movies,on="movieId")
movieRating["title"]=movieRating.title.str.split(" \(").str.get(0)
movieRating["genres"]=movieRating.genres.str.split("|").str.get(0)

In [ ]:
print "Number of Users: ",len(movieRating.userId.unique())
print "Number of Movies reviewed: ",len(movieRating.movieId.unique())

In [ ]:
lastDate=movieRating.timestamp.max()
firstDate=movieRating.timestamp.min()

In [ ]:
print "Start Date: ",datetime.fromtimestamp(firstDate)
print "End Date: ",datetime.fromtimestamp(lastDate)

Here we saw the time frame within which the ratings were given

In [ ]:
ratings_copy=ratings.iloc[:,:2]
connectedRating=ratings.iloc[:,:2].merge(ratings_copy,on=["movieId",ratings_copy.userId!=ratings.userId])
connectedRating=connectedRating[connectedRating.userId_x!=connectedRating.userId_y]
G=nx.Graph()
for line in connectedRating.itertuples():
    G.add_edge(line[1],line[3])

In [ ]:
nx.draw(G,edge_color='#A0CBE2',width=0.25,node_size=50)
plt.show()

This shows that all users have some movies in common that they have rated. Hence, we do not need to exclude any of the user now.

In [ ]:
topMovie=movieRating.title.value_counts()[:10]
tm=topMovie.plot(kind='bar')
plt.ylabel('# of time Rated')
plt.xlabel("Movie")
plt.show()

The above chart shows the top 10 movies were rated the highest number of times

In [ ]:
topGenre=movieRating.genres.value_counts()[:10]
tg=topGenre.plot(kind='bar')
plt.ylabel('# of time Rated')
plt.xlabel("Genre")
plt.show()

The above chart shows number of time each genre movie was rated.

Now we will look at some of the user features:

In [ ]:
df_user.head()

In [ ]:
print "Average age of users: ",user.age.mean()
print "Minimum age of user in data: ",user.age.min()
print "Maximum age of user in data: ",user.age.max()

In [ ]:
genderC=user.gender.value_counts()
gc=genderC.plot(kind='bar')
plt.ylabel('# of users')
plt.xlabel("Gender")
plt.show()

This graph higglights number of users in our data according to their gender.

In [ ]:
occC=user.occupation.value_counts()
oc=occC.plot(kind='barh')
plt.xlabel('# of users')
plt.ylabel("Occupation")
plt.show()

This graph shows the number of users according to their occupation.

In [ ]:
genderOccCategory=df_user[["occupation","gender","userId"]].groupby(by=["occupation","gender"]).count()
genderOccCategory.unstack().plot(kind="bar",stacked=True)
plt.ylabel('# of users')
plt.xlabel("Gender")
plt.show()

This graph helps us in understadning the ratio of Men to Women for each occupation in the data.
We need to clean the zip codes and segment users according to geographic location as well.
These infographics will help us in our model in later stages.

### Building the Recommendation Model 

#### Collaborative Filtering Overview

In collaberative-filtering, the matrix factorization method discovers latent features underlying the interactions between user and movie, assuming the inner product of user's latent features $u_i$ and movies latent $v_j$ is the rating:
$$h_\mathbf{\theta}(i,j) = \mathbf{u_i^T v_j}$$
Suppose there are $k$ latent features in $u$ and $v$. Then the model can be thought of describing how each latent feature $k$ matches to each other. For example, if the number of latent variables $k$ is 4:
$$\mathbf{u_i} = [0.4, 0.5, -0.3, 0.7]$$ $$\mathbf{v_j} = [0.2, -0.6, -0.2. 0.3]$$
The rating will be the inner product of these two vectors:
$$[0.4*0.2]_{feature1}+[0.5*(-0.6)]_{feature2}+[(-0.3)*(-0.2)]_{feature_3}+[0.7*0.3]_{feature4} = 0.05 $$

Then by minimizing the loss funtion, it is effectively finding the latent features and the coeficients that minimize the error of the reconstructed matrix compared to the training data.
$$\sum_{i,j\in S}\ell(h_\theta(i,j),\mathbf{X}_{ij}) + \lambda_u ||\mathbf{U}||_2^2 + \lambda_v ||\mathbf{V}||_2^2$$

#### Different Model

The motivation for this project is that there are some pitfalls with collaborative filtering: 
* It is hard to interpret the meaning of latent features.
* The inner product suggest that the rating is the linear combination of the latent features, which might not be true.
* There is a lot of information available that influences the rating which is not utilized. 

Instead of the collaborative filtering approach, the intuition that this project aims to follow is that each person is a regressor, where the input features are the defined movie attributes. In other word, it is equal to minimize the loss function for each person given rating history independently:
$$\sum_{j\in S}\ell(h_{\theta_i}(\mathbf{x}_j), \mathbf{X}_{ij}) $$, where i denotes a person and his/her preferences are encoded in parameter $\mathbf{\theta}_i$, $\mathbf{x}_j$ is the feature vector of the movie (with features such as genre, actors, year,etc...), and $h$ is the hypothesis function. For example, given a movie with features $m_j$={'1996', 'action, romance', ...}, a person with preferences $\theta_i$={'>1950', 'action || romance', ...} will output a rating value through a model $h$ (i.e. decision tree, SVM, cosine simlarity, ...).

#### 1. Linear Regression
Assume the rating is the output of the linear regression model:
$$h_{\theta_i}(\mathbf{x_j}) = \mathbf{\theta_i}^T\mathbf{x_j} $$, where $\theta_i$ is the user parameter and $x_j$ (k x 1) is the movie feature (k x 1).

The goal is to minimize the mean square error. For each user, the parameter could be solved by constructing a matrix $\mathbf{X}$ based on the past rating history, and it is a standard form of least square.
For example, if user i rated m movies, the equation will be: 

$$\mathbf{\theta_i^*} = \arg \min_{\theta} ||\mathbf{X\theta - y}||_2$$, where $\mathbf{X}$ is a m x k matrix and each row is a movie feature vector $\mathbf{x}$, $\mathbf{\theta_i}$ is a k x 1 column vector of user i's parameter.

In this experiement, three numeric feature {'year', 'runtime', 'vote'} are selected as the movie feature (i.e k = 3), and the number of user and movies are as same as the collaberative filter setting.

In [ ]:
np.random.seed(10)
# Collaberative Filtering
reload(cf)
X_tr, X_te, movieNames, idxToUserId, idxToLenId = cf.process(ratings, movies, np.random.permutation(len(ratings)))
U_cf, V_cf = cf.train(X_tr, X_te, k = 3, lam = 1, niters = 51, verbose = True)

In [ ]:
# Linear Regression
reload(lr)
V_lr = lr.process(imdb_movies, idxToLenId, links, features = ['year', 'votes', 'runtimes'])
U_lr = lr.train(X_tr, V_lr, lam = 1)

Below is the experiement result comparing with the collaberative filtering method as a baseline, using the some evaluation metric. Although the training error is higher, the testing error is actually lower than the collaberative filtering method. 

In [ ]:
# Evaluation
train_mse, test_mse = cf.error(X_tr, U_cf, V_cf), cf.error(X_te, U_cf, V_cf)
print "CF Train error: {:0.4f} Test error: {:0.4f}".format(train_mse, test_mse)

train_mse, test_mse = cf.error(X_tr, U_lr, V_lr), cf.error(X_te, U_lr, V_lr)
print "LR Train error: {:0.4f} Test error: {:0.4f}".format(train_mse, test_mse)

#### Experiment: one feature 

In [ ]:
# Year only
V_lr = lr.process(imdb_movies, idxToLenId, links, features = ['year'])
U_lr = lr.train(X_tr, V_lr, lam = 1)
train_mse, test_mse = cf.error(X_tr, U_lr, V_lr), cf.error(X_te, U_lr, V_lr)

print "Year only:"
print "LR Train error: {:0.4f} Test error: {:0.4f}".format(train_mse, test_mse)
print 

# Votes only
V_lr = lr.process(imdb_movies, idxToLenId, links, features = ['votes'])
U_lr = lr.train(X_tr, V_lr, lam = 1)
train_mse, test_mse = cf.error(X_tr, U_lr, V_lr), cf.error(X_te, U_lr, V_lr)

print 'Votes only:'
print "LR Train error: {:0.4f} Test error: {:0.4f}".format(train_mse, test_mse)
print

# Runtimes only
V_lr = lr.process(imdb_movies, idxToLenId, links, features = ['runtimes'])
U_lr = lr.train(X_tr, V_lr, lam = 1)
train_mse, test_mse = cf.error(X_tr, U_lr, V_lr), cf.error(X_te, U_lr, V_lr)

print 'Runtimes only:'
print "LR Train error: {:0.4f} Test error: {:0.4f}".format(train_mse, test_mse)
print

#### CF - k vs error

In [ ]:
np.random.seed(10)
# Collaberative Filtering
reload(cf)
X_tr, X_te, movieNames, idxToLenId = cf.process(ratings, movies, np.random.permutation(len(ratings)))
param_k = [1,3,5,10,20,50]
UV_cfs = [cf.train(X_tr, X_te, k = i, lam = 1, niters = 16, verbose = False) for i in param_k]

In [ ]:
plt.plot(param_k, [cf.error(X_tr, uv[0], uv[1]) for uv in UV_cfs], label = 'train')
plt.plot(param_k, [cf.error(X_te, uv[0], uv[1]) for uv in UV_cfs], label = 'test')
plt.title('k(#latent variable) vs error')
plt.xlabel('k')
plt.ylabel('mse')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

#### CF - iteration vs error

In [ ]:
param_it = [1,3,5,10,20,50]
UV_cfs = [cf.train(X_tr, X_te, k = 3, lam = 1, niters = i, verbose = False) for i in param_it]

In [ ]:
plt.plot(param_it, [cf.error(X_tr, uv[0], uv[1]) for uv in UV_cfs], label = 'train')
plt.plot(param_it, [cf.error(X_te, uv[0], uv[1]) for uv in UV_cfs], label = 'test')
plt.title('iter vs error')
plt.xlabel('iter')
plt.ylabel('mse')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

#### 2DHistogram (release year to rating)

In [ ]:
V_lr = lr.process(imdb_movies, idxToLenId, links, features = ['year', 'age', 'votes'])
rating_samples_tr = [(X_tr[i,j], V_lr[j]) for i, j in zip(X_tr.nonzero()[0], X_tr.nonzero()[1])]
rating_samples_te = [(X_te[i,j], V_lr[j]) for i, j in zip(X_te.nonzero()[0], X_te.nonzero()[1])]

target_tr, year_tr, age_tr, votes_tr = [i for i, j in rating_samples_tr], [j for i, j in rating_samples_tr]

target_tr, year_tr = [i for i, j in rating_samples_tr], [j for i, j in rating_samples_tr]


V_lr = lr.process(imdb_movies, idxToLenId, links, features = ['year'])

In [ ]:
# movieRating.merge(user, on='userId').merge(links, on='movieId').merge(imdb_movies, on='imdbId')

In [ ]:
from matplotlib.ticker import NullFormatter
from matplotlib.colors import LogNorm

# Define the locations for the axes
left, width = 0.1, 0.65
bottom, height = 0.1, 0.65
bottom_h = left_h = left + width + 0.02

# Set up the geometry of the three plots
rect_hist2d = [left, bottom, width, height]
rect_histx = [left, bottom_h, width, 0.2]
rect_histy = [left_h, bottom, 0.2, height]

# Set up the size of the figure
plt.figure(1, figsize=(16, 8))

# Make the three plots
axHist2D = plt.axes(rect_hist2d)
axHistx = plt.axes(rect_histx) # x histogram
axHisty = plt.axes(rect_histy) # y histogram

# Remove the inner axes numbers of the histograms
nullfmt = NullFormatter()
axHistx.xaxis.set_major_formatter(nullfmt)
axHisty.yaxis.set_major_formatter(nullfmt)

axHist2D.hist2d(y, x, cmap="hot")
axHistx.hist(x)
axHisty.hist(y, orientation='horizontal')

xbinwidth = 0.5
ybinwidth = 5
xmax, ymax = np.max(x), np.max(y)

xlim = (int(xmax/xbinwidth) + 1) * xbinwidth
ylim = (int(ymax/ybinwidth) + 1) * ybinwidth

xbins = np.arange(0, xlim + xbinwidth, xbinwidth) 
ybins = np.arange(0, ylim + ybinwidth, ybinwidth)

# bins = np.arange(-lim, lim + binwidth, binwidth)
axHistx.hist(y, bins = ybins)
axHisty.hist(x, bins = xbins, orientation='horizontal')

axHistx.set_xlim(axHist2D.get_xlim())
axHisty.set_ylim(axHist2D.get_ylim())

#### Decade histogram (Group by rating)

In [ ]:
df = pd.DataFrame({'rating' : x, 'year' : y})

df_test = pd.
df['decade'] = map(lambda x : (int(x) / 10) * 10, df['year'])

In [ ]:
groupByRating = df['year'].hist(by=df['rating'], figsize = (15, 15))

#### Rating histogram (Group by decade)

In [ ]:
groupByDecade = df['rating'].hist(by=df['decade'], figsize = (15, 15), bins = np.arange(0,6,0.5))

In [ ]:
import seaborn
seaborn.regplot(x = df['year'], y = df['rating'], scatter = True, fit_reg = True, x_estimator=np.mean)


In [ ]:
from scipy.stats import pearsonr
print 'linear correlation: ', pearsonr(df['year'], df['rating'])[0]

In [ ]:
reload(process_data)
df_movies = movie_parsing.parse(df_movies)

In [ ]:
process_data.process(df_movies, df_user)

### References

F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4, Article 19 (December 2015), 19 pages. DOI=http://dx.doi.org/10.1145/2827872